# Breakdown of Source Code Changes

This notebook generates the table for a breakdown of source code changes.


In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

from typing import Dict, Tuple
from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import load_pkl, save_latex, mini_bar, center_cell

data: DiffResult = load_pkl("src")

KINDS = {
    DepKind.FUNC: DepKind.FUNC,
    DepKind.STRUCT: DepKind.STRUCT,
    DepKind.TRACEPOINT: "Tracept",
}

GROUPS = {VersionGroup.LTS}

ISSUES_IGNORE = [
    IssueEnum.ADD,
    IssueEnum.REMOVE,
    IssueEnum.OLD,
    IssueEnum.NEW,
]


table: Dict[Tuple[str, str], Dict[Tuple[DepKind, str], str]] = {}
for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue

    for pair, pair_result in group_result.iter_pairs():
        col = {}

        for kind, kind_result in pair_result.iter_kinds():
            if kind not in KINDS:
                continue
            change = kind_result.issues[IssueEnum.CHANGE]
            for issue, count in kind_result.iter_issues():
                if issue in ISSUES_IGNORE:
                    continue
                if count == 0:
                    continue
                if kind == DepKind.TRACEPOINT:
                    if issue not in (
                        IssueEnum.CHANGE,
                        IssueEnum.TRACE_EVENT_CHANGE,
                        IssueEnum.TRACE_FUNC_CHANGE,
                    ):
                        continue

                if issue == IssueEnum.CHANGE:
                    issue = "No. changed"
                    text = f"{count}"
                    text = f"{count / 1000:.1f}k" if count > 1000 else str(count)
                else:
                    issue = f" - {issue}"
                    text = mini_bar(
                        text=f"{count / change * 100:.0f}%",
                        percent=count / change,
                        total_width=1.15,
                        color="change",
                        bg_color="bg",
                    )

                col[(kind, issue)] = text

        table[(group.to_str(pair.v1), group.to_str(pair.v2))] = col

df = pd.DataFrame(
    {
        (
            "Linux Kernel Version",
            center_cell(f"{v1} - {v2}"),
        ): {(KINDS[kind], issue): text for (kind, issue), text in col.items()}
        for (v1, v2), col in table.items()
    }
)
latex = df.to_latex(multicolumn_format="c", column_format="cl|rrrr")
save_latex(latex, "breakdown")
df

[ utils_pickle.py:18 ] INFO: Loding src from /users/szhong/DepSurf/output/src.pkl
[  utils_latex.py:118] INFO: Saved breakdown to /users/szhong/DepSurf/paper/tabs/breakdown.tex


Linux Kernel Version                                                                                                                                                                                                                                                                                                
                                                                                                 \multicolumn{1}{c}{4.4 - 4.15}                                                                  \multicolumn{1}{c}{4.15 - 5.4}                                                                  \multicolumn{1}{c}{5.4 - 5.15}                                                                  \multicolumn{1}{c}{5.15 - 6.8}
Function No. changed                                                                                                       2.0k                                                                                            1.8k                                                                                            2.2k                                                                                            3.0k
          - Param added          \colorbox{bg}{\makebox[{0.49cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.66cm}][r]{57%}}  \colorbox{bg}{\makebox[{0.54cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.61cm}][r]{53%}}  \colorbox{bg}{\makebox[{0.46cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.69cm}][r]{60%}}  \colorbox{bg}{\makebox[{0.56cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.59cm}][r]{51%}}
          - Param removed        \colorbox{bg}{\makebox[{0.68cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.47cm}][r]{41%}}  \colorbox{bg}{\makebox[{0.73cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.42cm}][r]{36%}}  \colorbox{bg}{\makebox[{0.64cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.51cm}][r]{44%}}  \colorbox{bg}{\makebox[{0.60cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.55cm}][r]{48%}}
          - Param reordered      \colorbox{bg}{\makebox[{0.93cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.22cm}][r]{19%}}  \colorbox{bg}{\makebox[{0.93cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.22cm}][r]{19%}}  \colorbox{bg}{\makebox[{0.86cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.29cm}][r]{25%}}  \colorbox{bg}{\makebox[{0.94cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.21cm}][r]{19%}}
          - Param type changed   \colorbox{bg}{\makebox[{0.85cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.30cm}][r]{26%}}  \colorbox{bg}{\makebox[{0.88cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.27cm}][r]{23%}}  \colorbox{bg}{\makebox[{0.85cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.30cm}][r]{26%}}  \colorbox{bg}{\makebox[{0.86cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.29cm}][r]{25%}}
          - Return type changed  \colorbox{bg}{\makebox[{0.98cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.17cm}][r]{15%}}  \colorbox{bg}{\makebox[{0.91cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.24cm}][r]{21%}}  \colorbox{bg}{\makebox[{1.00cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.15cm}][r]{13%}}  \colorbox{bg}{\makebox[{0.96cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.19cm}][r]{17%}}
Struct   No. changed                                                                                                       1.1k                                                                                            1.1k                                                                                            1.3k                                                                                            1.5k
          - Field added          \colorbox{bg}{\makebox[{0.33cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.82cm}][r]{72%}}  \colorbox{bg}{\makebox[{0.30cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.85cm}][r]{74%}}  \colorbox{bg}{\makebox[{0.29cm}][r]{\phantom{0}}}\colorbox{change}{\makebox[{0.86cm}][r]{75%}}  \colorbox{bg}{\makebox[{0.30cm}][r]{\phant